<a href="https://colab.research.google.com/github/akash-kaul/ecosys/blob/master/SyntheaGraphViz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code-Installations

In [ ]:
!pip install python-igraph
!pip install plotly
!pip install plotly.express
!pip install pytigergraph

import plotly.express as px
import plotly.graph_objects as go
import igraph as ig
import pyTigerGraph as tg

#Graph Code

In [ ]:
graph = tg.TigerGraphConnection(
                host="https://synthea.i.tgcloud.io",
                graphname="MedGraph",
                apiToken="0g18ehjsq00pkcke6m0pc673o7rjalfs"
)

selectAll = graph.runInstalledQuery("grab_All_3d_demo", sizeLimit=40000000)

patients = selectAll[0]['data']
other = selectAll[1]['other']
edges = selectAll[2]['@@edgeList']
links = []

l = len(edges)
# for i in range(l):
#     if edges[i]['to_type'] == "Address":
#         links.append((edges[i]['from_id'], edges[i]['to_id']))
#     if edges[i]['to_type'] == "Allergies":
#         links.append((edges[i]['from_id'], edges[i]['to_id']))
#     if edges[i]['to_type'] == "ImagingStudies":
#         links.append((edges[i]['from_id'], edges[i]['to_id']))

links = [(edges[i]['from_id'], edges[i]['to_id']) for i in range(l) if edges[i]['to_type'] == "Address" or edges[i]['to_type'] == "Allergies" or edges[i]['to_type'] == "ImagingStudies"]

G = ig.Graph()

labels = []
group = []
hover = []

for i in range(len(patients)):
    patient_last = patients[i]['attributes']['lastName']
    patient_first = patients[i]['attributes']['firstName']
    hover.append(patient_first + " " + patient_last)
    labels.append(patients[i]['v_id'])
    group.append(0)

for i in range(len(other)):
    if other[i]['v_type'] == 'Address':
        labels.append(other[i]['v_id'])
        hover.append(other[i]['v_id'])
        group.append(1)
    elif other[i]['v_type'] == 'ImagingStudies':
        labels.append(other[i]['v_id'])
        hover.append(other[i]['attributes']['bodySiteDescription'] + ", " + other[i]['attributes']['modalityDescription'])
        group.append(2)
    elif other[i]['v_type'] == 'Allergies':
        labels.append(other[i]['v_id'])
        hover.append(other[i]['attributes']['description'])
        group.append(3)

G.add_vertices(labels)
G.add_edges(links)


layt = G.layout('fr_3d')
# kk_3d, fr_3d, random_3d, circular_3d

print(layt[5])

N = len(labels)

Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates

Xe=[]
Ye=[]
Ze=[]
for e in links:
    w = labels.index(e[0])
    z = labels.index(e[1])
    Xe+=[layt[w][0],layt[z][0], None]# x-coordinates of edge ends
    Ye+=[layt[w][1],layt[z][1], None]
    Ze+=[layt[w][2],layt[z][2], None]
  

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(0,0,0)', width=3),
               hoverinfo='none'
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
              #  name='people',
               marker=dict(symbol='circle',
                             size=6,
                             color=group,
                             colorscale='sunset',
                             line=dict(color='rgb(50,50,50)', width=1.0)
                             ),
               text=labels,
               hovertemplate=['{}<extra></extra>'.format(hover[i]) for i in range(len(hover))]
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Healthcare System",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
        template="plotly_white",
     margin=dict(
        t=100
    ),
    hovermode='closest',
    # annotations=[
    #        dict(
    #        showarrow=False,
    #         text="",
    #         xref='paper',
    #         yref='paper',
    #         x=0,
    #         y=0.1,
    #         xanchor='left',
    #         yanchor='bottom',
    #         font=dict(
    #         size=14
    #         )
    #         )
    #     ],    
)

data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)

fig.show()

[18.875837303048755, 40.487226856797776, 21.949261957133565]
